In [26]:
import torch
import torch.optim as optim
import numpy as np
from ddn.pytorch.node import *

def f(x):
    return (1 - x[0])**2 + 100 * (x[1] - x[0]**2)**2

# L-BFGS
x_lbfgs = 10*torch.ones(2, 2)
x_lbfgs.requires_grad = True

optimizer = optim.LBFGS([x_lbfgs],
                        history_size=10,
                        max_iter=4,
                        line_search_fn="strong_wolfe")
print(x_lbfgs)

h_lbfgs = []
for i in range(100):
    optimizer.zero_grad()
    objective = f(x_lbfgs)
    objective.backward()
    optimizer.step(lambda: f(x_lbfgs))
    h_lbfgs.append(objective.item())
    
print(x_lbfgs)

tensor([[10.],
        [10.]], requires_grad=True)
tensor([[1.0000],
        [1.0000]], requires_grad=True)


In [43]:
# This semi works, but the solution is wrong and the gradient crashes half way

import autograd.numpy as np
import torch
from autograd import grad, jacobian

def gradient(f, x, y):
    fY = grad(f,1)
    fYY = jacobian(fY, 1)
    fXY = jacobian(fY, 0)
    
    return -1.0 * np.linalg.solve(fYY(x,y), fXY(x,y))

n = 5
M = np.random.uniform(0,255,(n,n))
symm = M@M.T
# test for symmetry
print(symm)

torch.set_default_tensor_type(torch.FloatTensor)
x = torch.tensor(symm, requires_grad=True)

D = x.sum(0).diag() # D is an NxN diagonal matrix with d on diagonal, for d(i) = sum_j(w(i,j))
ONE = torch.ones(x.size(dim=0),1)   # Nx1 vector of all ones
L = D - x

# L.backward(x)

L.t()


x = torch.tensor([[0,1,0,0], [1,0,0,3], [0,0,0,0], [0,3,0,0]]).double()
D = x.sum(0).diag()
print(D)

x1 = torch.tensor([[[0,1,0,0], [1,0,0,3], [0,0,0,0], [0,3,0,0]]]).double()
D = torch.einsum('bij->bj', x1)
d1, d2 = D.size()
D = torch.diag_embed(D)
print(D)

[[ 48599.46576081  38373.48733538  42606.22737027  64382.48332668
   75880.33838495]
 [ 38373.48733538 108485.22082652  77075.26240585  56840.58479613
   88266.94769604]
 [ 42606.22737027  77075.26240585 120540.05088788  76841.17040939
  141603.78912639]
 [ 64382.48332668  56840.58479613  76841.17040939 113480.37148607
  134061.21810102]
 [ 75880.33838495  88266.94769604 141603.78912639 134061.21810102
  198665.55956067]]
tensor([[1., 0., 0., 0.],
        [0., 4., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 3.]], dtype=torch.float64)
tensor([[[1., 0., 0., 0.],
         [0., 4., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 3.]]], dtype=torch.float64)


In [63]:
import torch
import torch.optim as optim
import numpy as np
from ddn.pytorch.node import *

# class NormalizedCuts(EqConstDeclarativeNode):
class NormalizedCuts(AbstractDeclarativeNode):
    def __init__(self):
        super().__init__()
        
    def general_eigen(self, A, y):
        """ f = y^T A y """
        
        # Batch         
        yT = torch.einsum('bij->bji', y)
        # Batch matrix multiplication
        return torch.einsum('bij,bjk->bik', torch.einsum('bij,bjk->bik', yT, A), y)
        
        # For single problem...        
        # return torch.matmul(torch.matmul(y.t()), A), y)
    
    def objective(self, x, y):
        """ f(x,y) = y^T (D-W) y """
        D = torch.einsum('bij->bj', x1)
        D = torch.diag_embed(D)
        L = D - x # Laplacian matrix
        return self.general_eigen(L, y)
        
#     def equality_constraints(self, x, y):
#         """ h(x,y) = y^T y = 1 """
#         return torch.matmul(y.t(), y) - 1
        
    def solve(self, x):
        x.detach()
        y = torch.rand_like(x, requires_grad=True)
        y = self._run_optimisation( x, y=y)
        return y.detach(), None
    
    def _run_optimisation(self, *xs, y):
            with torch.enable_grad():
                opt = torch.optim.LBFGS([y],
                                        lr=1.0,
                                        max_iter=1000,
                                        max_eval=None,
                                        tolerance_grad=1e-40,
                                        tolerance_change=1e-40,
                                        history_size=100,
                                        line_search_fn="strong_wolfe"
                                        )
                def reevaluate():
                    opt.zero_grad()
                    f = self.objective(*xs, y=y).sum() # sum over batch elements
                    f.backward()
                    return f
                opt.step(reevaluate)
            return y
        
torch.set_default_tensor_type(torch.DoubleTensor)

node = NormalizedCuts()
x = torch.tensor([[[0,1,0,0], [1,0,0,3], [0,0,0,0], [0,3,0,0]]]).double()
print(x)
print(x.size())
y,_ = node.solve(x)
print(y)
node.gradient(x, y=y)

tensor([[[0., 1., 0., 0.],
         [1., 0., 0., 3.],
         [0., 0., 0., 0.],
         [0., 3., 0., 0.]]])
torch.Size([1, 4, 4])
tensor([[[0.6662, 0.3332, 1.1314, 0.1954],
         [0.6891, 0.7011, 0.8247, 0.1113],
         [0.6618, 0.0142, 0.4685, 0.2312],
         [0.8381, 0.2516, 0.2792, 0.9572]]])


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn